In [1]:
import os

# 1. Create a directory for the data
if not os.path.exists('data'):
    os.makedirs('data')

# 2. Download the zip file (using a reliable mirror for the NASA dataset)
!wget https://physics.bu.edu/~pankajm/ML-Notebooks/files/CMAPSSData.zip -O data/CMAPSSData.zip

# 3. Unzip it
!unzip -o data/CMAPSSData.zip -d data/

print("Data downloaded and extracted!")

--2025-12-03 19:18:23--  https://physics.bu.edu/~pankajm/ML-Notebooks/files/CMAPSSData.zip
Resolving physics.bu.edu (physics.bu.edu)... 128.197.41.42
Connecting to physics.bu.edu (physics.bu.edu)|128.197.41.42|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-12-03 19:18:24 ERROR 404: Not Found.

Archive:  data/CMAPSSData.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of data/CMAPSSData.zip or
        data/CMAPSSData.zip.zip, and cannot find data/CMAPSSData.zip.ZIP, period.
Data downloaded and extracted!


In [2]:
# The dataset has 26 columns
# 1. Unit Number (Which engine is it?)
# 2. Time Cycles (How long has it been running?)
# 3-5. Operational Settings (Altitude, Speed, etc.)
# 6-26. Sensor Readings (s1 to s21)

index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i) for i in range(1, 22)] 
col_names = index_names + setting_names + sensor_names

print(col_names)

['unit_nr', 'time_cycles', 'setting_1', 'setting_2', 'setting_3', 's_1', 's_2', 's_3', 's_4', 's_5', 's_6', 's_7', 's_8', 's_9', 's_10', 's_11', 's_12', 's_13', 's_14', 's_15', 's_16', 's_17', 's_18', 's_19', 's_20', 's_21']


In [ ]:
import pandas as pd

#train_df = pd.read_csv('data/train_FD001.txt', sep=r'\s+', header=None, names=col_names)
train_df = pd.read_csv('data/train_FD001.txt', sep=r' +', header=None, names=col_names)

print("Training data shape:", train_df.shape)
train_df.head()

Training data shape: (20631, 26)


,unit_nr,time_cycles,setting_1,setting_2,setting_3,s_1,s_2,s_3,s_4,s_5,...,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
